In [69]:
import numpy as np
import csv

#LOAD DATA CSV menjadi array of dataset dan array of label
def load_csv(filename):
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        line_count = 0
        #MAKE DUMMY ARRAY
        X = np.zeros((1,13))
        label = np.array(None)
        for row in csv_reader:
            if line_count == 0:
                #NOT USING THE COLUMN NAMES
#                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                #ADD ELEMENT AND LABEL TO ARRAY
                line = np.empty(0)
                for i in range(13):
                    if row[i] == "?":
                        line = np.append(line, None)
                    else:
                        line = np.append(line, row[i])
                X = np.append(X, np.array([line]), axis=0)
                label = np.append(label, row[13])
                line_count += 1
        #REMOVE DUMMY ELEMENT
        X = np.delete(X, 0, 0)
        label = np.delete(label, 0)
    return X, label

def sort_X(X):
    arr = [[] for i in range(len(X[0]))]
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] != None:
                arr[j].append(X[i][j])
    for i in range(len(X[0])):
        arr[i].sort()
    return arr

def exchange_question_mark(X, arr):
    for i in range(len(X)):
        for j in range(len(X[i])):
            if X[i][j] == None:
                X[i][j] = arr[j][len(arr[j])//2]
    return X

def normalize(X):
    for i in range(len(X)):
        for j in range(len(X[i])):
            if j == 0:
                X[i][j] = float(X[i][j]) / 50
            elif j == 3:
                X[i][j] = float(X[i][j]) / 100
            elif j == 4:
                X[i][j] = float(X[i][j]) / 100
            elif j == 7:
                X[i][j] = float(X[i][j]) / 100
    return X

X1, label1 = load_csv('train.csv')
arr1 = sort_X(X1)
X1 = exchange_question_mark(X1, arr1)
X1 = normalize(X1)

In [70]:
X2, label2 = load_csv('train.csv')
X3, label3 = load_csv('test.csv')
X2 = np.append(X2, X3, axis=0)
label2 = np.append(label2, label3)
arr2 = sort_X(X2)
X2 = exchange_question_mark(X2, arr2)

In [71]:
from sklearn.cluster import AgglomerativeClustering
clustering_model_1 = AgglomerativeClustering(linkage='single').fit(X1)
clustering_model_2 = AgglomerativeClustering(linkage='single').fit(X2)
print(clustering_model_1.labels_)
print(clustering_model_2.labels_)

[1 1 1 ... 1 1 1]
[0 0 0 ... 0 0 0]


In [72]:
#ASUMSI labelclust dan label ukuran sama
def purity(labelclust, label):
    labelmap = {}
    summax = 0
    for i in range(len(labelclust)):
        if labelclust[i] not in labelmap.keys():
            labelmap[labelclust[i]] = {}
        if label[i] not in labelmap[labelclust[i]].keys():
            labelmap[labelclust[i]][label[i]] = 1
        else:
            labelmap[labelclust[i]][label[i]] += 1
    for el in labelmap.keys():
        summax += max(labelmap[el].values())
    return summax/len(labelclust)
                
print(purity(clustering_model_1.labels_, label))
print(purity(clustering_model_2.labels_, label2))

0.8333333333333334
0.8238944918541505


In [73]:
from sklearn.cluster import DBSCAN
clustering_DBSCAN_model_1 = DBSCAN().fit(X1)
clustering_DBSCAN_model_2 = DBSCAN().fit(X2)
print(purity(clustering_DBSCAN_model_1.labels_, label))
print(purity(clustering_DBSCAN_model_2.labels_, label2))

0.8925
0.8231186966640807
